In [3]:
# Este notebook fue creado en colab, por lo tanto esta optimizado para que acceda a los archivos de Drive

from google.colab import drive
drive.mount("/content/drive")

Mounted at /content/drive


In [6]:
%cd "/content/drive/My Drive/UNI/COLABS/"

# importar todas las librerias necesarias

from keras.layers import Conv2D, Conv2DTranspose, UpSampling2D
from keras.layers import Activation, Dense, Dropout, Flatten, InputLayer
from tensorflow.keras.layers import BatchNormalization
from keras.callbacks import TensorBoard
from keras.models import Sequential, model_from_json
from keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.utils import array_to_img, img_to_array, load_img
from skimage.color import rgb2lab, lab2rgb, rgb2gray
from skimage.io import imsave
import numpy as np
import os
import random
import tensorflow as tf

from sklearn.metrics import mean_squared_error
from PIL import Image, ImageOps
import matplotlib.pyplot as plt

from colorization.colorizers import *
from statistics import mean

/content/drive/My Drive/UNI/COLABS


In [1]:
%cd

/root


In [ ]:
X = [] # Crear archivo donde esten todas las imágenes como matrices

In [ ]:
for filename in os.listdir('/content/drive/My Drive/UNI/DATA/natural_color/color'): # Por cada imagen de color
  X.append(img_to_array(load_img('/content/drive/My Drive/UNI/DATA/natural_color/color/'+filename))) # Se agrega como matriz

In [ ]:
print(len(X)) # Imprimir cantidad de imagenes

688


In [ ]:
X = np.array(X, dtype=float) # Guardar como arreglo de numpy

In [ ]:
split = int(0.95*len(X)) # Dividir el 95% para entrenamiento y 5% para pruebas
Xtrain = X[:split] # Todas hasta split
Xtrain = 1.0/255*Xtrain # Normalizar

In [ ]:
print(Xtrain.shape) # 653 imagenes de entrenamiento

(653, 256, 256, 3)


In [ ]:
model = Sequential() # Instanciar modelo de keras secuencial para que se sumen los modelos
model.add(InputLayer(input_shape=(256, 256, 1))) # Ajustar la entrada para aceptar una matriz de 256 * 256 pixeles
model.add(Conv2D(64, (3, 3), activation='relu', padding='same')) # Todos los bloques convolucionales
model.add(Conv2D(64, (3, 3), activation='relu', padding='same', strides=2))
model.add(Conv2D(128, (3, 3), activation='relu', padding='same'))
model.add(Conv2D(128, (3, 3), activation='relu', padding='same', strides=2))
model.add(Conv2D(256, (3, 3), activation='relu', padding='same'))
model.add(Conv2D(256, (3, 3), activation='relu', padding='same', strides=2))
model.add(Conv2D(512, (3, 3), activation='relu', padding='same'))
model.add(Conv2D(256, (3, 3), activation='relu', padding='same'))
model.add(Conv2D(128, (3, 3), activation='relu', padding='same'))
model.add(UpSampling2D((2, 2)))
model.add(Conv2D(64, (3, 3), activation='relu', padding='same'))
model.add(UpSampling2D((2, 2)))
model.add(Conv2D(32, (3, 3), activation='relu', padding='same'))
model.add(Conv2D(2, (3, 3), activation='tanh', padding='same'))
model.add(UpSampling2D((2, 2)))
model.compile(optimizer='rmsprop', loss='mse') # Compilar el modelo y establecer las métricas

In [ ]:
datagen = ImageDataGenerator(
        shear_range=0.2,
        zoom_range=0.2,
        rotation_range=20,
        horizontal_flip=True)

# Instancia de ImageDataGenerator que genera grupos de tensor de imágenes con aumentación de datos

batch_size = 10

In [ ]:
def image_a_b_gen(batch_size):  
    for batch in datagen.flow(Xtrain, batch_size=batch_size): # Generar los batches
        lab_batch = rgb2lab(batch) # Pasar de rgb a lab
        X_batch = lab_batch[:,:,:,0] # Separar l
        Y_batch = lab_batch[:,:,:,1:] / 128 # Separar ab
        yield (X_batch.reshape(X_batch.shape+(1,)), Y_batch) # Reconfigurar la matriz para tener el tamaño de la entrada

In [ ]:
tensorboard = TensorBoard(log_dir="/content/drive/My Drive/UNI/AM-PROY-2/Full-version/first_run") # Instanciar TensorBoard para visualización
model.fit_generator(image_a_b_gen(batch_size), callbacks=[tensorboard], epochs=10, steps_per_epoch=950) # Entrenar el modelo con los batches de imágenes

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  


Epoch 1/10
950/950 [==============================] - 227s 227ms/step - loss: 0.5996
Epoch 2/10
950/950 [==============================] - 216s 227ms/step - loss: 0.0220
Epoch 3/10
950/950 [==============================] - 216s 227ms/step - loss: 0.0146
Epoch 4/10
950/950 [==============================] - 215s 226ms/step - loss: 0.0123
Epoch 5/10
950/950 [==============================] - 215s 226ms/step - loss: 0.0105
Epoch 6/10
950/950 [==============================] - 214s 226ms/step - loss: 0.0094
Epoch 7/10
950/950 [==============================] - 215s 227ms/step - loss: 0.0088
Epoch 8/10
950/950 [==============================] - 214s 225ms/step - loss: 0.0078
Epoch 9/10
950/950 [==============================] - 212s 223ms/step - loss: 0.0073
Epoch 10/10
950/950 [==============================] - 207s 218ms/step - loss: 0.0064


In [ ]:
# Si se entrenó, se corre esta celda para guardar el modelo

model_json = model.to_json() # json guarda la estructura de los bloques convolusionales
with open("/content/drive/My Drive/UNI/AM-PROY-2/Beta-version/model2.json", "w") as json_file:
    json_file.write(model_json)
model.save_weights("/content/drive/My Drive/UNI/AM-PROY-2/Beta-version/model_3.h5") # Guarda los pesos entrenados de las neuronas

In [ ]:
# Si se quiere recuperar el modelo sin entrenar, se corre esta celda

json_file = open("/content/drive/My Drive/UNI/AM-PROY-2/Beta-version/model2.json", 'r')
loaded_model_json = json_file.read()
json_file.close()
model = model_from_json(loaded_model_json) # Cargar la estructura de los bloques convolucionales

model.load_weights("/content/drive/My Drive/UNI/AM-PROY-2/Beta-version/model_3.h5") # Cargar los pesos del modelo

model.compile(optimizer='rmsprop', loss='mse') # Compilar el modelo

In [ ]:
Xtest = rgb2lab(1.0/255*X[split:])[:,:,:,0] # Imagenes de prueba en lab separados solo l
Xtest = Xtest.reshape(Xtest.shape+(1,)) # Reconfigurar para que tenga el mismo tamaño que la entrada de la red neuronal
Ytest = rgb2lab(1.0/255*X[split:])[:,:,:,1:] # Imagenes de prueba en lab, solamente ab
Ytest = Ytest / 128 # Dividir entre 128 para normalizar y que sea a y b de 0 a 1.

In [ ]:
print(len(Xtest))
print(len(Ytest))
type(Xtest)

35
35


numpy.ndarray

In [ ]:
batch_size = 10 # MSE del modelo en Keras

print(model.evaluate(Xtest, Ytest, batch_size=batch_size))
print(model.evaluate(Xtest, Ytest, batch_size = 1))

4/4 [==============================] - 3s 620ms/step - loss: 0.0032
0.003204912645742297
35/35 [==============================] - 3s 90ms/step - loss: 0.0032
0.003204912878572941


In [ ]:
color_me = [] # Arreglo vacío para guardar las matrices de las imágenes
for filename in os.listdir('/content/drive/My Drive/UNI/DATA/natural_color/gray'): # Cargar las imágenes
  color_me.append(img_to_array(load_img('/content/drive/My Drive/UNI/DATA/natural_color/gray/'+filename)))

color_me = X[split:] # Reescribir para que sean las mismas que pruebas
print(len(color_me))
color_me = np.array(color_me, dtype=float) # Modificar a flotantes por si hay enteros
color_me = rgb2lab(1.0/255*color_me)[:,:,:,0] # Cambiar a lab, pero solo obtener l
color_me = color_me.reshape(color_me.shape+(1,)) # Reconfigurar para que tenga el tamaño de la entrada

35


In [ ]:
output = model.predict(color_me) # Predecir por cada imágen en el arreglo
output = output * 128 # Multiplicar por 128 para regresar de la normalización

2/2 [==============================] - 3s 224ms/step


In [ ]:
for i in range(len(output)): # Por cada imágen que regrese el modelo
  cur = np.zeros((256, 256, 3)) # Arreglo vacio para juntar la entrada con el output del modelo
  cur[:,:,0] = color_me[i][:,:,0]
  cur[:,:,1:] = output[i]
  imsave(f"/content/drive/My Drive/UNI/COLABS/pred/Testing_{i}.png", lab2rgb(cur)) # Guardar como imagen el resultado

/usr/local/lib/python3.7/dist-packages/skimage/color/colorconv.py:1109: UserWarning: Color data out of range: Z < 0 in 4 pixels
  return xyz2rgb(lab2xyz(lab, illuminant, observer))
/usr/local/lib/python3.7/dist-packages/skimage/color/colorconv.py:1109: UserWarning: Color data out of range: Z < 0 in 19 pixels
  return xyz2rgb(lab2xyz(lab, illuminant, observer))
/usr/local/lib/python3.7/dist-packages/skimage/color/colorconv.py:1109: UserWarning: Color data out of range: Z < 0 in 2 pixels
  return xyz2rgb(lab2xyz(lab, illuminant, observer))
/usr/local/lib/python3.7/dist-packages/skimage/color/colorconv.py:1109: UserWarning: Color data out of range: Z < 0 in 12 pixels
  return xyz2rgb(lab2xyz(lab, illuminant, observer))
/usr/local/lib/python3.7/dist-packages/skimage/color/colorconv.py:1109: UserWarning: Color data out of range: Z < 0 in 9 pixels
  return xyz2rgb(lab2xyz(lab, illuminant, observer))
/usr/local/lib/python3.7/dist-packages/skimage/color/colorconv.py:1109: UserWarning: Color da

In [ ]:
%cd /content/drive/My Drive/UNI/COLABS/ # Cambiar la ruta

/root


In [7]:
colorizer_siggraph17 = siggraph17(pretrained=True).eval() # Cargar el modelo entrenado

Downloading: "https://colorizers.s3.us-east-2.amazonaws.com/siggraph17-df00044c.pth" to /root/.cache/torch/hub/checkpoints/siggraph17-df00044c.pth


  0%|          | 0.00/130M [00:00<?, ?B/s]

In [ ]:
Y = np.zeros((35, 256, 256, 3)) # Arreglo vacio que va a guardar las imágenes de prueba

for i in range(len(X[split:])): # Por cada imagen de prueba
  x = X[split:][i] # Guardar la imagen
  img = array_to_img(x) # Transformarla a matriz
  img.save(f'original/Testing_{i}.png') # Guardar la imagen original
  img = ImageOps.grayscale(img) # Transformarla a escala de grises
  img.save(f'bw/Testing_{i}.png') # Guardar la imágen en escala de grises
  img = load_img(f'bw/Testing_{i}.png') # Cargar la imagen

  # Pasarla por el colorizador siggraph
  (tens_l_orig, tens_l_rs) = preprocess_img(img, HW=(256,256))
  img_bw = postprocess_tens(tens_l_orig, torch.cat((0*tens_l_orig,0*tens_l_orig),dim=1))
  out_img_siggraph17 = postprocess_tens(tens_l_orig, colorizer_siggraph17(tens_l_rs).cpu())
  # out_img_siggraph17 matriz que predice el modelo de siggraph

  plt.imsave(f'algorithm_siggraph/Testing_{i}.png', out_img_siggraph17) # Guardar la predicción como imagen
  Y[i] = out_img_siggraph17 # Guardar el resultado como matriz

In [10]:
def rgb_ab( tup ):
  if type(tup) == int: # En caso de que la función regrese getpixel regrese un 
                       # solo número, se transforma en tupla rgb con ese mismo 
                       # valor en los 3
    tup = (tup, tup, tup)
  # Función que transforma rgb a ab con las fórmulas del paper
  # Acepta una tupla con 3 valores rgb y regresa una tupla de 2 (a,b)
  I = (tup[0] + tup[1] + tup[2])/3
  if (I == 0):
    return (0,0)
  a = (tup[2]/I) - ((tup[0] + tup[1])/(2*I))
  b = (tup[0] - tup[1])/I
  return (a,b)

In [13]:
%cd

/root


In [11]:
# importar nombres de las imágenes de cada directorio
images_bw = os.listdir('/content/drive/My Drive/UNI/COLABS/bw')
images_sig = os.listdir('/content/drive/My Drive/UNI/COLABS/algorithm_siggraph')
images_color = os.listdir('/content/drive/My Drive/UNI/COLABS/original')
images_pred = os.listdir('/content/drive/My Drive/UNI/COLABS/pred')

# Arreglo que contendrá el MSE de cada imágen
MSE_a = []
MSE_b = []

for i in range(len(images_color)): # Por cada imagen
  img_color = Image.open('/content/drive/My Drive/UNI/COLABS/original/' + images_color[i]) # Abrir imagen a color
  img_pred = Image.open('/content/drive/My Drive/UNI/COLABS/pred/' + images_pred[i]) # Abrir predicción

  # Se guardan los valores de los pixeles 
  Y_pred_a = []
  Y_pred_b = []
  Y_true_a = []
  Y_true_b = []

  for w in range(img_color.size[0]):
    for h in range(img_color.size[1]):
      # Por cada pixel en altura y ancho de la imagen
        # Regresar el valor a,b del pixel y agregarlo a su respectiva lista entre las imagenes original y la predicción:
        Y_pred_a.append(rgb_ab(img_pred.getpixel((w,h)))[0])
        Y_pred_b.append(rgb_ab(img_pred.getpixel((w,h)))[1])  
        Y_true_a.append(rgb_ab(img_color.getpixel((w,h)))[0])
        Y_true_b.append(rgb_ab(img_color.getpixel((w,h)))[1])
  
  # Guardar el valor final de MSE de toda la imagen en la respectiva lista
  MSE_a.append(mean_squared_error(Y_true_a,Y_pred_a))
  MSE_b.append(mean_squared_error(Y_true_b,Y_pred_b))

# Sacar el promedio de las listas
print("MSE promedio por imagen entre color y nuestras predicciones")
print("a", mean(MSE_a))
print("b", mean(MSE_b))

MSE promedio por imagen entre color y nuestras predicciones
a 0.06090106495983092
b 0.011591519796284158


In [ ]:
# Arreglo que contendrá el MSE de cada imágen
MSE_a = []
MSE_b = []

for i in range(len(images_color)): # Por cada imagen
  img_color = Image.open('/content/drive/My Drive/UNI/COLABS/original/' + images_color[i]) # Abrir imagen a color
  img_sigg = Image.open('/content/drive/My Drive/UNI/COLABS/algorithm_siggraph/' + images_pred[i]) # Abrir predicción

  # Se guardan los valores de los pixeles 
  Y_pred_a = []
  Y_pred_b = []
  Y_true_a = []
  Y_true_b = []

  for w in range(img_color.size[0]):
    for h in range(img_color.size[1]):
      # Por cada pixel en altura y ancho de la imagen
        # Regresar el valor a,b del pixel y agregarlo a su respectiva lista entre las imagenes original y la predicción:
        Y_pred_a.append(rgb_ab(img_sigg.getpixel((w,h)))[0])
        Y_pred_b.append(rgb_ab(img_sigg.getpixel((w,h)))[1])  
        Y_true_a.append(rgb_ab(img_color.getpixel((w,h)))[0])
        Y_true_b.append(rgb_ab(img_color.getpixel((w,h)))[1])
  
  # Guardar el valor final de MSE de toda la imagen en la respectiva lista
  MSE_a.append(mean_squared_error(Y_true_a,Y_pred_a))
  MSE_b.append(mean_squared_error(Y_true_b,Y_pred_b))

# Sacar el promedio de las listas
print("MSE promedio por imagen entre color y algoritmo siggraph")
print("a", mean(MSE_a))
print("b", mean(MSE_b))

MSE promedio por imagen entre color y algoritmo siggraph
a 0.09295181503978404
b 0.009430649917240436


In [12]:
# Arreglo que contendrá el MSE de cada imágen
MSE_a = []
MSE_b = []

for i in range(len(images_color)): # Por cada imagen
  img_color = Image.open('/content/drive/My Drive/UNI/COLABS/original/' + images_color[i]) # Abrir imagen a color
  img_bw = Image.open('/content/drive/My Drive/UNI/COLABS/bw/' + images_pred[i]) # Abrir predicción

  # Se guardan los valores de los pixeles 
  Y_pred_a = []
  Y_pred_b = []
  Y_true_a = []
  Y_true_b = []

  for w in range(img_color.size[0]):
    for h in range(img_color.size[1]):
      # Por cada pixel en altura y ancho de la imagen
        # Regresar el valor a,b del pixel y agregarlo a su respectiva lista entre las imagenes original y la predicción:
        Y_pred_a.append(rgb_ab(img_bw.getpixel((w,h)))[0])
        Y_pred_b.append(rgb_ab(img_bw.getpixel((w,h)))[1])  
        Y_true_a.append(rgb_ab(img_color.getpixel((w,h)))[0])
        Y_true_b.append(rgb_ab(img_color.getpixel((w,h)))[1])
  
  # Guardar el valor final de MSE de toda la imagen en la respectiva lista
  MSE_a.append(mean_squared_error(Y_true_a,Y_pred_a))
  MSE_b.append(mean_squared_error(Y_true_b,Y_pred_b))

# Sacar el promedio de las listas
print("MSE promedio por imagen entre color y escala de grises")
print("a", mean(MSE_a))
print("b", mean(MSE_b))

MSE promedio por imagen entre color y escala de grises
a 0.29198281505354023
b 0.028430336858659706
